In [0]:
import urllib.request
urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/spark_proj1_sfcrime/data/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/spark_proj1_sfcrime/data/"))

path,name,size
dbfs:/spark_proj1_sfcrime/data/sf_03_18.csv,sf_03_18.csv,559169754


In [0]:
data_path = "dbfs:/spark_proj1_sfcrime/data/sf_03_18.csv"

In [0]:
# Import package 
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
#from ggplot import *
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [0]:
# Data Preprocessing
crime_data_lines = sc.textFile(data_path)

--------------------------------------------------------------------------- 
 Exception Traceback (most recent call last)
 <command-2767661930650828> in <module> 
 1 # Data Preprocessing 
 2 crime_data_lines = sc . textFile ( data_path ) 
 ----> 3 display ( crime_data_lines ) 

 /local_disk0/tmp/1600834078342-0/PythonShell.py in display (self, input, *args, **kwargs) 
 1437 self . displayHTML ( input . _repr_html_ ( ) ) 
 1438 else : 
 -> 1439 raise Exception ( genericErrorMsg ) 
 1440 
 1441 def displayHTML ( self , html ) : 

 Exception : Cannot call display(<class 'pyspark.rdd.RDD'>)
 Call help(display) for more info.

In [0]:
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'location', 'SF Find Neighborhoods 2 2', 'Current Police Districts 2 2', 'Current Supervisor Districts 2 2', 'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2', 'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2', 'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2', 'DELETE - 2017 Fix It Zones 2 2', 'Civic Center Harm Reduction Project Boundary 2 2', 'Fix It Zones as of 2017-11-06 2 2', 'DELETE - HSOC Zones 2 2', 'Fix It Zones as of 2018-02-07 2 2', 'CBD, BID and GBD Boundaries as of 2017 2 2', 'Areas of Vulnerability, 2016 2 2', 'Central Market/Tenderloin Boundary 2 2', 'Central Market/Tenderloin Boundary Polygon - Updated 2 2', 'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2', 'Neighborhoods 2']

In [0]:
#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

In [0]:
#get the total number of data 
print(crimes.count())

2160953

In [0]:
crimes

Out[10]: PythonRDD[16] at RDD at PythonRDD.scala:58

In [0]:
# Get dataframe and sql

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)
display(df_opt1)
df_opt1.createOrReplaceTempView("sf_crime")

PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,location,SF Find Neighborhoods 2 2,Current Police Districts 2 2,Current Supervisor Districts 2 2,Analysis Neighborhoods 2 2,DELETE - Fire Prevention Districts 2 2,DELETE - Police Districts 2 2,DELETE - Supervisor Districts 2 2,DELETE - Zip Codes 2 2,DELETE - Neighborhoods 2 2,DELETE - 2017 Fix It Zones 2 2,Civic Center Harm Reduction Project Boundary 2 2,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
11087941463010,110879414,63010,WARRANTS,WARRANT ARREST,Friday,07/01/2011,08:00,NORTHERN,"ARREST, BOOKED",3400 Block of PIERCE ST,-122.439757857075,37.8021507619169,POINT (-122.43975785707501 37.8021507619169),17,4,6,13,13,9,1,57,17,null,null,null,null,null,null,1,null,null,null,null,17
5117770316010,051177703,16010,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Tuesday,10/18/2005,14:30,TENDERLOIN,"ARREST, BOOKED",0 Block of UNITEDNATIONS PZ,-122.414317857881,37.7799444052046,POINT (-122.414317857881 37.7799444052046),21,5,10,36,7,10,9,28852,36,3,1,3,1,3,6,2,1,1,1,39,21
5011209707055,050112097,07055,VEHICLE THEFT,RECOVERED VEHICLE - STOLEN OUTSIDE SF,Saturday,01/29/2005,13:45,BAYVIEW,NONE,1500 Block of KIRKWOOD AV,-122.388798895151,37.7375755833256,POINT (-122.38879889515101 37.7375755833256),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86
11044468164020,110444681,64020,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Thursday,06/02/2011,02:52,CENTRAL,PSYCHOPATHIC CASE,900 Block of COLUMBUS AV,-122.414354301151,37.8031089840376,POINT (-122.41435430115101 37.8031089840376),107,6,3,32,3,1,10,308,32,null,null,null,null,null,null,1,null,null,null,null,107
3038311107024,030383111,07024,VEHICLE THEFT,STOLEN TRAILER,Saturday,02/01/2003,08:00,BAYVIEW,NONE,1500 Block of BAY SHORE BL,-122.401096851568,37.7245556697717,POINT (-122.401096851568 37.7245556697717),86,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null,86
6018621406243,060186214,06243,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Sunday,02/12/2006,17:00,CENTRAL,NONE,400 Block of DAVIS CT,-122.398187664281,37.7967148853927,POINT (-122.39818766428101 37.7967148853927),77,6,3,8,3,1,10,28860,6,null,null,null,null,null,null,1,null,null,null,null,77
9606640606374,096066406,06374,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Thursday,08/06/2009,22:00,RICHMOND,NONE,2000 Block of BAKER ST,-122.444143357971,37.7899090887789,POINT (-122.44414335797102 37.7899090887789),102,8,6,30,13,6,1,29490,27,null,null,null,null,null,null,1,null,null,null,null,102
10037601606304,100376016,06304,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Friday,04/23/2010,16:30,MISSION,NONE,2600 Block of MISSION ST,-122.41860030589,37.7546255872838,POINT (-122.41860030589001 37.7546255872838),53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null,53
5118607706244,051186077,06244,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Thursday,10/20/2005,11:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.40340479147902 37.775420706711),32,1,10,34,14,2,9,28853,34,null,null,null,null,null,null,2,null,null,null,null,32
12038512205053,120385122,05053,BURGLARY,"BURGLARY OF STORE, UNLAWFUL ENTRY",Thursday,05/03/2012,10:00,SOUTHERN,NONE,600 Block of 2ND ST,-122.39123835272,37.7812532736369,POINT (-122.39123835272001 37.7812532736369),31,1,10,8,6,2,9,28856,6,null,null,null,null,null,null,1,null,null,null,null,31


In [0]:
df_opt1.printSchema()

root
-- PdId: string (nullable = true)
-- IncidntNum: string (nullable = true)
-- Incident Code: string (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- Date: string (nullable = true)
-- Time: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: string (nullable = true)
-- Y: string (nullable = true)
-- location: string (nullable = true)
-- SF Find Neighborhoods 2 2: string (nullable = true)
-- Current Police Districts 2 2: string (nullable = true)
-- Current Supervisor Districts 2 2: string (nullable = true)
-- Analysis Neighborhoods 2 2: string (nullable = true)
-- DELETE - Fire Prevention Districts 2 2: string (nullable = true)
-- DELETE - Police Districts 2 2: string (nullable = true)
-- DELETE - Supervisor Districts 2 2: string (nullable = true)
-- DELETE - Zip Codes 2 2: string (nullable = true)
-- DELETE - Neighborhoods 2 2: string (nullable = true)
-- DELETE - 2017 Fix It Zones 2 2: string (nullable = true)
-- Civic Center Harm Reduction Project Boundary 2 2: string (nullable = true)
-- Fix It Zones as of 2017-11-06 2 2: string (nullable = true)
-- DELETE - HSOC Zones 2 2: string (nullable = true)
-- Fix It Zones as of 2018-02-07 2 2: string (nullable = true)
-- CBD, BID and GBD Boundaries as of 2017 2 2: string (nullable = true)
-- Areas of Vulnerability, 2016 2 2: string (nullable = true)
-- Central Market/Tenderloin Boundary 2 2: string (nullable = true)
-- Central Market/Tenderloin Boundary Polygon - Updated 2 2: string (nullable = true)
-- HSOC Zones as of 2018-06-05 2 2: string (nullable = true)
-- OWED Public Spaces 2 2: string (nullable = true)
-- Neighborhoods 2: string (nullable = true)

In [0]:
%sql
select count(*)
from sf_crime

count(1)
2160953


In [0]:
## find number of crimes in different categories

# dataframe based solution
q1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)
display(q1_result)

category,count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


In [0]:
# SQL based solution
crime_category_num = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crime_category_num)

category,Count
LARCENY/THEFT,477975
OTHER OFFENSES,303027
NON-CRIMINAL,236937
ASSAULT,191384
VEHICLE THEFT,126228
DRUG/NARCOTIC,117875
VANDALISM,114718
WARRANTS,99821
BURGLARY,91067
SUSPICIOUS OCC,79740


In [0]:
## Count number of crimes of different districts
crimeDistrict = spark.sql("SELECT PdDistrict, COUNT(*) as Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC")
display(crimeDistrict)

PdDistrict,Count
SOUTHERN,394234
MISSION,293072
NORTHERN,269229
CENTRAL,223962
BAYVIEW,210729
TENDERLOIN,189034
INGLESIDE,186645
TARAVAL,158710
PARK,121070
RICHMOND,114267


In [0]:
## Count number of crimes each "Sunday" at "SF downtown"
q3_result = spark.sql("""
                      with Sunday_dt_crime as(
                      select substring(Date,1,5) as Date,
                             substring(Date,7) as Year
                      from sf_crime
                      where (DayOfWeek = 'Sunday'
                             and -122.423671 <= X
                             and X <= 122.412497
                             and 37.773510 <= Y
                             and Y <= 37.782137)
                             )
                             
                      select Year, Date, COUNT(*) as Count
                      from Sunday_dt_crime
                      group by Year, Date
                      order by Year, Date
                      """)
display(q3_result)

Year,Date,Count
2003,01/05,28
2003,01/12,33
2003,01/19,19
2003,01/26,32
2003,02/02,45
2003,02/09,46
2003,02/16,50
2003,02/23,49
2003,03/02,40
2003,03/09,49


In [0]:
## Analysis number of crime in each month of 2015-2018
q4_result = spark.sql("""
                      with Monthly_crime as(
                      select Date,
                             substring(Date,7) as Year,
                             substring(Date,1,2) as Month
                      from sf_crime
                      )

                      select Month, COUNT(*) as Count
                      from Monthly_crime
                      where Year in ('2015', '2016', '2017', '2018')
                      group by Month
                      order by Month
                      """)
display(q4_result)


Month,Count
01,50851
02,45666
03,49831
04,47598
05,42567
06,37376
07,38043
08,38458
09,37373
10,39122


In [0]:
q4_result2 = spark.sql("""
                      with Monthly_crime as(
                      select Date,
                             substring(Date,7) as Year,
                             substring(Date,1,2) as Month
                      from sf_crime
                      )
                      
                      select Year, Month, COUNT(*) as Count
                      from Monthly_crime
                      where Year in ('2015', '2016', '2017', '2018')
                      group by Year, Month
                      order by Year, Month
                      """)
display(q4_result2)

Year,Month,Count
2015,01,13370
2015,02,12055
2015,03,13674
2015,04,12705
2015,05,13511
2015,06,13073
2015,07,13158
2015,08,13523
2015,09,12666
2015,10,12889


In [0]:
## show number of crimes by hour 
q5_result = spark.sql("""
                      select substring(Time,1,2) as Hour,
                      count(*) as Count
                      from sf_crime
                      group by Hour
                      order by Hour
                      """)
display(q5_result)

Hour,Count
00,111449
01,64426
02,53963
03,35271
04,24997
05,22178
06,32898
07,53103
08,79464
09,86795


In [0]:
# Show number of crime by hour for records in Christmas
q5_result_ch = spark.sql("""
                      select substring(Time,1,2) as Hour,
                      count(*) as Count
                      from sf_crime
                      where Date like '12/25/%'
                      group by Hour
                      order by Hour
                      """)
display(q5_result_ch)

Hour,Count
00,215
01,138
02,114
03,79
04,60
05,49
06,57
07,62
08,110
09,83


In [0]:
# Show number of crime by hour for records in New Year Eve (12/31 - 01/01)
q5_result_ne = spark.sql("""
                      select substring(Time,1,2) as Hour,
                             substring(Date,1,5) as Date_in_year,
                             count(*) as Count
                      from sf_crime
                      where Date like '12/31/%' or Date like '01/01/%'
                      group by Date_in_year, Hour
                      order by Date_in_year desc, Hour
                      """)
display(q5_result_ne)

Hour,Date_in_year,Count
00,12/31,286
01,12/31,162
02,12/31,138
03,12/31,72
04,12/31,58
05,12/31,59
06,12/31,72
07,12/31,98
08,12/31,179
09,12/31,173


In [0]:
## (1) Step1: Find out the top-3 danger disrict
## (2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1
## (3) give your advice to distribute the police based on your analysis results.

In [0]:
crimeDistrict.createOrReplaceTempView("crimeDistrict")

Dist_rank = spark.sql("""
                      select PdDistrict, Count as Crime_Number,
                             rank() over(order by Count desc) as Rank
                      from crimeDistrict
                      """)
Dist_rank.createOrReplaceTempView("Dist_rank")

q6_s1_result = spark.sql("""
                       select PdDistrict, Crime_Number
                       from Dist_rank
                       where Rank <= 3
                       """)
q6_s1_result.createOrReplaceTempView("q6_s1_result")

display(q6_s1_result)

PdDistrict,Crime_Number
SOUTHERN,394234
MISSION,293072
NORTHERN,269229


In [0]:
# Step 2 - Category

Top3_Dist = spark.sql("""
                      select sf_crime.PdDistrict, sf_crime.category, sf_crime.Time
                      from sf_crime
                      inner join q6_s1_result on sf_crime.PdDistrict = q6_s1_result.PdDistrict
                      """)
Top3_Dist.createOrReplaceTempView("Top3_Dist")

q6_s2_result1 = spark.sql("""
                         select category,
                                count(*) as Crime_Number
                         from Top3_Dist
                         group by category
                         order by Crime_Number desc
                         """)
display(q6_s2_result1)

category,Crime_Number
LARCENY/THEFT,242382
OTHER OFFENSES,127563
NON-CRIMINAL,107688
ASSAULT,80080
DRUG/NARCOTIC,50300
WARRANTS,48628
VANDALISM,45591
VEHICLE THEFT,43482
BURGLARY,36364
SUSPICIOUS OCC,31554


In [0]:
# larceny/theft
q6_s2_result2 = spark.sql("""
                         select substring(Time,1,2) as Hour,
                                count(*) as Crime_Number
                         from Top3_Dist
                         where category = 'LARCENY/THEFT'
                         group by Hour
                         order by Hour
                         """)
display(q6_s2_result2)

Hour,Crime_Number
00,10082
01,6644
02,4065
03,2419
04,1542
05,1490
06,2478
07,3780
08,6278
09,7661


In [0]:
# burglary
q6_s2_result2 = spark.sql("""
                         select substring(Time,1,2) as Hour,
                                count(*) as Crime_Number
                         from Top3_Dist
                         where category = 'BURGLARY'
                         group by Hour
                         order by Hour
                         """)
display(q6_s2_result2)

Hour,Crime_Number
00,1585
01,1064
02,1152
03,1229
04,1032
05,948
06,858
07,998
08,1669
09,1379


In [0]:
# assault
q6_s2_result2 = spark.sql("""
                         select substring(Time,1,2) as Hour,
                                count(*) as Crime_Number
                         from Top3_Dist
                         where category = 'ASSAULT'
                         group by Hour
                         order by Hour
                         """)
display(q6_s2_result2)


Hour,Crime_Number
00,4675
01,4310
02,3837
03,1846
04,1127
05,889
06,1216
07,1743
08,2578
09,2885


In [0]:
## for different categories of crime, find the percentage of resolution
res_num = spark.sql("""select category, resolution, count(*) as N_res from sf_crime group by category, resolution""")
res_num.createOrReplaceTempView("res_num")

cate_num = spark.sql("""select category, count(*) as N_cate from sf_crime group by category""")
cate_num.createOrReplaceTempView("cate_num")

q7_result = spark.sql("""
                      select distinct sf_crime.category, sf_crime.resolution, N_res/N_cate as Percentage
                      from (sf_crime left join res_num on sf_crime.category = res_num.category and sf_crime.resolution = res_num.resolution)
                      left join cate_num on sf_crime.category = cate_num.category
                      order by category, resolution""")
q7_result.createOrReplaceTempView("q7_result")
# Percentage of resolution for LARCENY/THEFT
q7 = spark.sql("""
               select Resolution, Percentage
               from q7_result
               where category = 'LARCENY/THEFT'
               order by Percentage desc
               """)
display(q7)

Resolution,Percentage
NONE,0.9162131910664784
"ARREST, BOOKED",0.052588524504419684
"ARREST, CITED",0.021890266227313143
NOT PROSECUTED,0.0033474554108478475
UNFOUNDED,0.0025545269104032637
COMPLAINANT REFUSES TO PROSECUTE,9.854071865683352E-4
DISTRICT ATTORNEY REFUSES TO PROSECUTE,9.770385480412154E-4
EXCEPTIONAL CLEARANCE,9.247345572467179E-4
PROSECUTED BY OUTSIDE AGENCY,3.2637690255766513E-4
PSYCHOPATHIC CASE,9.833150269365553E-5


In [0]:
# Percentage of resolution for BURGLARY
q7 = spark.sql("""
               select Resolution, Percentage
               from q7_result
               where category = 'BURGLARY'
               order by Percentage desc
               """)
display(q7)

Resolution,Percentage
NONE,0.8417209307432989
"ARREST, BOOKED",0.14017152206617106
UNFOUNDED,0.005984604741563903
DISTRICT ATTORNEY REFUSES TO PROSECUTE,0.004535122492230995
COMPLAINANT REFUSES TO PROSECUTE,0.0032723159871303545
"ARREST, CITED",0.0016032152151712476
EXCEPTIONAL CLEARANCE,0.0014714441015955286
NOT PROSECUTED,6.039509372220453E-4
PROSECUTED BY OUTSIDE AGENCY,3.513896362019173E-4
LOCATED,1.6471389196964873E-4


In [0]:
# Percentage of resolution for ASSAULT
q7 = spark.sql("""
               select Resolution, Percentage
               from q7_result
               where category = 'ASSAULT'
               order by Percentage desc
               """)
display(q7)

Resolution,Percentage
NONE,0.5943391297078126
"ARREST, BOOKED",0.32818312920620324
"ARREST, CITED",0.03750052250971868
DISTRICT ATTORNEY REFUSES TO PROSECUTE,0.013647953851941647
COMPLAINANT REFUSES TO PROSECUTE,0.011108556619153116
UNFOUNDED,0.006661998913179785
EXCEPTIONAL CLEARANCE,0.0038456715294904486
PSYCHOPATHIC CASE,0.0022415666931404924
NOT PROSECUTED,0.0011442962839108806
PROSECUTED BY OUTSIDE AGENCY,7.315136061530745E-4
